<a href="https://colab.research.google.com/github/suarez-oa/Proyecto_Final_Henry/blob/main/Oa_Suarez_Merchan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import json
import numpy as np
import pickle
import pyarrow.parquet as pq
import os
from google.cloud import storage
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Definir la configuración de Google Cloud
project_id = 'oa-suarez-prueba'
bucket_name = 'oa_suarez_merchan'
estados_folder = 'estados'

# Configura el proyecto
!gcloud config set project {project_id} -q

# Crea una instancia de Client para Google Cloud Storage
client = storage.Client()

# Accede al bucket de Google Cloud Storage
bucket = client.get_bucket(bucket_name)


Updated property [core/project].


In [ ]:
def normalize_categories(category_list):
    if category_list is not None:
        return [category.strip() for category in category_list]
    else:
        return None


def normalize_hours(hours_list):
    if hours_list is not None:
        normalized_hours = []
        for day_hours in hours_list:
            if len(day_hours) == 2:
                day, hours = day_hours
                normalized_day_hours = f"{day}: {hours}"
                normalized_hours.append(normalized_day_hours)
        return normalized_hours
    else:
        return None


def flatten_dict(d):
    if d is None:
        return {}

    flattened_dict = {}
    for key, values in d.items():
        if isinstance(values, list):
            for value in values:
                flattened_dict[f"{key}: {value}"] = 1
        else:
            flattened_dict[key] = values
    return flattened_dict



def cargar_y_normalizar(ruta_archivo):
    with open(ruta_archivo, 'r') as f:
        j = [json.loads(line) for line in f]
    df = pd.DataFrame(j)
    df['category'] = df['category'].apply(normalize_categories)
    df_meta = df.dropna(subset=['category'])
    df_meta = df_meta.explode('category')
    df_meta = df_meta.drop(columns=['relative_results'])
    df_meta = df_meta.drop(columns=['MISC'])
    df_meta['hours'] = df_meta['hours'].apply(normalize_hours)
    df_meta['hours'] = df_meta['hours'].apply(lambda x: ', '.join(x) if x is not None else None)
    return df_meta


def limpiar_datos(line):
    try:
        # Reemplazar caracteres de escape por espacios en blanco
        line = line.replace('\\', ' ')
        return json.loads(line)
    except json.JSONDecodeError:
        return None


In [ ]:
#selecciono la categoria que deseo buscar en la MetaData
categoria_especifica = 'Tourist attraction'

ruta_archivo1 = '/content/drive/MyDrive/Oa_Suarez/Google Maps/metadata-sitios/1.json'
ruta_archivo2 = '/content/drive/MyDrive/Oa_Suarez/Google Maps/metadata-sitios/2.json'
ruta_archivo3 = '/content/drive/MyDrive/Oa_Suarez/Google Maps/metadata-sitios/3.json'
ruta_archivo4 = '/content/drive/MyDrive/Oa_Suarez/Google Maps/metadata-sitios/4.json'
ruta_archivo5 = '/content/drive/MyDrive/Oa_Suarez/Google Maps/metadata-sitios/5.json'
ruta_archivo6 = '/content/drive/MyDrive/Oa_Suarez/Google Maps/metadata-sitios/6.json'
ruta_archivo7 = '/content/drive/MyDrive/Oa_Suarez/Google Maps/metadata-sitios/7.json'
ruta_archivo8 = '/content/drive/MyDrive/Oa_Suarez/Google Maps/metadata-sitios/8.json'
ruta_archivo9 = '/content/drive/MyDrive/Oa_Suarez/Google Maps/metadata-sitios/9.json'
ruta_archivo10 = '/content/drive/MyDrive/Oa_Suarez/Google Maps/metadata-sitios/10.json'
ruta_archivo11 = '/content/drive/MyDrive/Oa_Suarez/Google Maps/metadata-sitios/11.json'

df_meta1 = cargar_y_normalizar(ruta_archivo1)
df_meta2 = cargar_y_normalizar(ruta_archivo2)
df_meta3 = cargar_y_normalizar(ruta_archivo3)
df_meta4 = cargar_y_normalizar(ruta_archivo4)
df_meta5 = cargar_y_normalizar(ruta_archivo5)
df_meta6 = cargar_y_normalizar(ruta_archivo6)
df_meta7 = cargar_y_normalizar(ruta_archivo7)
df_meta8 = cargar_y_normalizar(ruta_archivo8)
df_meta9 = cargar_y_normalizar(ruta_archivo9)
df_meta10 = cargar_y_normalizar(ruta_archivo10)
df_meta11 = cargar_y_normalizar(ruta_archivo11)

df_meta_n1 = df_meta1[df_meta1['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n2 = df_meta2[df_meta2['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n3 = df_meta3[df_meta3['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n4 = df_meta4[df_meta4['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n5 = df_meta5[df_meta5['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n6 = df_meta6[df_meta6['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n7 = df_meta7[df_meta7['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n8 = df_meta8[df_meta8['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n9 = df_meta9[df_meta9['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n10 = df_meta10[df_meta10['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n11 = df_meta11[df_meta11['category'].str.contains(categoria_especifica, case=False, na=False)]

#se concatena toda la meta data
df_total_meta = pd.concat([df_meta_n1, df_meta_n2, df_meta_n3, df_meta_n4, df_meta_n5, df_meta_n6, df_meta_n7, df_meta_n8, df_meta_n9, df_meta_n10, df_meta_n11], ignore_index=True)
df_total_meta = df_total_meta.fillna(0)
columnas_deseadas = ['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude', 'category', 'avg_rating', 'num_of_reviews', 'price']
df_extraido = df_total_meta[columnas_deseadas]
df_extraido.fillna(0, inplace=True)

#se guarda el MetaData en CSV con la categoria seleccionada
df_extraido.to_csv('/content/drive/MyDrive/Oa_Suarez/metadata_PW.csv', index=False)

#guardamos otro CSV mas pequeños para el Machine Learning
columnas_seleccionadas = ['gmap_id', 'price', 'num_of_reviews', 'avg_rating']
df_seleccionado_ml = df_extraido[columnas_seleccionadas]
df_seleccionado_ml.to_csv('/content/drive/MyDrive/Oa_Suarez/MetaData_ML.csv', index=False)

# Sube el primer archivo CSV a Google Cloud Storage
blob = bucket.blob('Metadata_PowerBI.csv')
blob.upload_from_filename('/content/drive/MyDrive/Oa_Suarez/metadata_PW.csv')

# Sube el segundo archivo CSV a Google Cloud Storage
blob2 = bucket.blob('MetaData_ML.csv')
blob2.upload_from_filename('/content/drive/MyDrive/Oa_Suarez/MetaData_ML.csv')


In [34]:
df_meta1.columns

Index(['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
       'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'state',
       'url'],
      dtype='object')

In [31]:
categoria_especifica = 'Pharmacy'

df_meta_n1 = df_meta1[df_meta1['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n2 = df_meta2[df_meta2['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n3 = df_meta3[df_meta3['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n4 = df_meta4[df_meta4['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n5 = df_meta5[df_meta5['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n6 = df_meta6[df_meta6['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n7 = df_meta7[df_meta7['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n8 = df_meta8[df_meta8['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n9 = df_meta9[df_meta9['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n10 = df_meta10[df_meta10['category'].str.contains(categoria_especifica, case=False, na=False)]
df_meta_n11 = df_meta11[df_meta11['category'].str.contains(categoria_especifica, case=False, na=False)]

#se concatena toda la meta data
df_total_meta = pd.concat([df_meta_n1, df_meta_n2, df_meta_n3, df_meta_n4, df_meta_n5, df_meta_n6, df_meta_n7, df_meta_n8, df_meta_n9, df_meta_n10, df_meta_n11], ignore_index=True)
df_total_meta = df_total_meta.fillna(0)
columnas_deseadas = ['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude', 'category', 'avg_rating', 'num_of_reviews', 'price']
df_extraido = df_total_meta[columnas_deseadas]
df_extraido.fillna(0, inplace=True)

#se guarda el MetaData en CSV con la categoria seleccionada
df_extraido.to_csv('/content/drive/MyDrive/Oa_Suarez/metadata_PW.csv', index=False)

#guardamos otro CSV mas pequeños para el Machine Learning
columnas_seleccionadas = ['gmap_id', 'price', 'num_of_reviews', 'avg_rating']
df_seleccionado_ml = df_extraido[columnas_seleccionadas]
df_seleccionado_ml.to_csv('/content/drive/MyDrive/Oa_Suarez/MetaData_ML.csv', index=False)

# Sube el primer archivo CSV a Google Cloud Storage
blob = bucket.blob('Metadata_PowerBI.csv')
blob.upload_from_filename('/content/drive/MyDrive/Oa_Suarez/metadata_PW.csv')

# Sube el segundo archivo CSV a Google Cloud Storage
blob2 = bucket.blob('MetaData_ML.csv')
blob2.upload_from_filename('/content/drive/MyDrive/Oa_Suarez/MetaData_ML.csv')

<ipython-input-31-d3e9f37c9493>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extraido.fillna(0, inplace=True)


In [ ]:
# Lista de directorios de los 50 estados
directorios = [
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Alabama',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Alaska',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Arizona',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Arkansas',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-California',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Colorado',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Connecticut',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Delaware',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-District_of_Columbia',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Florida',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Georgia',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Hawaii',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Idaho',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Illinois',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Indiana',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Iowa',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Kansas',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Kentucky',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Maine',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Maryland',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Massachusetts',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Michigan',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Minnesota',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Mississippi',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Missouri',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Montana',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Nebraska',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Nevada',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-New_Hampshire',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-New_Jersey',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-New_Mexico',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-New_York',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-North_Carolina',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-North_Dakota',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Ohio',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Oklahoma',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Oregon',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Pennsylvania',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Rhode_Island',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-South_Carolina',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-South_Dakota',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Tennessee',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Texas',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Utah',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Vermont',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Virginia',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Washington',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-West_Virginia',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Wisconsin',
    '/content/drive/MyDrive/Oa_Suarez/Google Maps/reviews-estados/review-Wyoming'

]

# Procesar y guardar los datos en archivos CSV
for directorio in directorios:
    df_estado = pd.DataFrame(columns=['user_id', 'name', 'time', 'rating', 'text', 'gmap_id'])

    for archivo_json in os.listdir(directorio):
        if archivo_json.endswith('.json'):
            ruta_completa = os.path.join(directorio, archivo_json)
            with open(ruta_completa, 'r', encoding='utf-8') as f:
                datos = [limpiar_datos(line) for line in f if line.strip()]
                datos = [d for d in datos if d is not None]  # Eliminar datos nulos
                df_temp = pd.DataFrame(datos)
                df_temp = df_temp[['user_id', 'name', 'time', 'rating', 'text', 'gmap_id']]
                df_estado = pd.concat([df_estado, df_temp], ignore_index=True)

    # Guardar el DataFrame en un archivo CSV correspondiente al directorio
    nombre_csv = directorio.split("/")[-1] + '.csv'
    df_estado.to_csv(nombre_csv, index=False)

    # Subir el archivo CSV a Google Cloud Storage
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(os.path.join(estados_folder, nombre_csv))
    blob.upload_from_filename(nombre_csv)


Updated property [core/project].


In [ ]:
# recreation

# Entertainment

In [32]:
#selecciono la categoria en Yelp que seo buscar y analizar
categoria_especifica_yelp = 'recreation'


# -----------------------------------------------------------------------------------------------------------------------

archivo_pkl = '/content/drive/MyDrive/Oa_Suarez/Yelp/business.pkl'
with open(archivo_pkl, 'rb') as archivo:
    business = pickle.load(archivo)
business = business.loc[:, ~business.columns.duplicated()]
columnas_seleccionadas = ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'categories']
business = business[columnas_seleccionadas]

business = business[business['categories'].str.contains(categoria_especifica_yelp, case=False, na=False)]

with open ('/content/drive/MyDrive/Oa_Suarez/Yelp/checkin.json','r') as f:
  jcheckin = [json.loads(line) for line in f]

df_checkin_Yelp= pd.DataFrame(jcheckin)

combined_df = df_checkin_Yelp.merge(business, on='business_id', how='inner')

#guardo el documento en CSV
combined_df.to_csv('/content/drive/My Drive/Oa_Suarez/Yelp_Checkin_Business.csv', index=False)

# creamos un CSV mas pequeño para el Machine Learning de Business y Checkin
columnas_seleccionadas_bc = ['business_id', 'date', 'state', 'is_open']
df_seleccionado_bc = combined_df[columnas_seleccionadas_bc]
df_seleccionado_bc.to_csv('/content/drive/My Drive/Oa_Suarez/Yelp_Checkin_Business-ML.csv', index=False)

# Sube el primer archivo CSV a Google Cloud Storage
blob3 = bucket.blob('Yelp_Checkin_Business_PowerBI.csv')
blob3.upload_from_filename('/content/drive/My Drive/Oa_Suarez/Yelp_Checkin_Business.csv')

# Sube el segundo archivo CSV a Google Cloud Storage
blob4 = bucket.blob('Yelp_Checkin_Business-ML.csv')
blob4.upload_from_filename('/content/drive/My Drive/Oa_Suarez/Yelp_Checkin_Business-ML.csv')




# ----------------------------------------------------------------------------------------------------------------------


review = '/content/drive/MyDrive/Oa_Suarez/Yelp/review.json'
df_review = pd.DataFrame()

with open(review, 'r', encoding='utf-8') as f:
    datos = [limpiar_datos(line) for line in f if line.strip()]
    datos = [d for d in datos if d is not None]  # Eliminar datos nulos
    df_temp = pd.DataFrame(datos)
    columnas_seleccionadas = ['review_id', 'user_id', 'business_id', 'useful', 'text']
    df_temp['text'] = df_temp['text'].astype(str)
    df_temp = df_temp.replace('', pd.NA)
    df_temp = df_temp[columnas_seleccionadas]

    # Rellenar valores nulos con ceros
    df_temp.fillna(0, inplace=True)

    df_review = pd.concat([df_review, df_temp], ignore_index=True)

#combinamos desde la tabla businnes los reviwes q tenga el mismo business_id de ellos
combined_df_business_reviwes = df_review.merge(business, on='business_id', how='inner')
combined_df_business_reviwes.to_csv('/content/drive/My Drive/Oa_Suarez/Yelp_Review_Business.csv', index=False, encoding='utf-8')

cambio_Ml = combined_df_business_reviwes

# creamos un CSV mas pequeño para el Machine Learning de Business y Checkin
columnas_a_mantener = ['business_id', 'state', 'stars', 'text']
cambio_Ml = cambio_Ml[columnas_a_mantener]
cambio_Ml.to_csv('/content/drive/My Drive/Oa_Suarez/Yelp_Reviwes_Business-ML.csv', index=False)

# Sube el primer archivo CSV a Google Cloud Storage
blob5 = bucket.blob('Yelp_Review_Business_PowerBI.csv')
blob5.upload_from_filename('/content/drive/My Drive/Oa_Suarez/Yelp_Review_Business.csv')

# Sube el segundo archivo CSV a Google Cloud Storage
blob6 = bucket.blob('Yelp_Reviwes_Business-ML.csv')
blob6.upload_from_filename('/content/drive/My Drive/Oa_Suarez/Yelp_Reviwes_Business-ML.csv')




#----------------------------------------------------------------------------------------------------------------------------------


tip = '/content/drive/MyDrive/Oa_Suarez/Yelp/tip.json'
df_tip = pd.DataFrame()

with open(tip, 'r', encoding='utf-8') as f:
    datos = [limpiar_datos(line) for line in f if line.strip()]
    datos = [d for d in datos if d is not None]  # Eliminar datos nulos
    df_temp = pd.DataFrame(datos)
    columnas_seleccionadas = ['user_id', 'business_id', 'text']
    df_temp = df_temp[columnas_seleccionadas]

    # Rellenar valores nulos con ceros
    df_temp.fillna(0, inplace=True)

    df_tip = pd.concat([df_tip, df_temp], ignore_index=True)

#combino la tabla tip con business para saber que datos necesito de tip
combined_df_business_tip = df_tip.merge(business, on='business_id', how='inner')

#lo guardo en un CSV
combined_df_business_tip.to_csv('/content/drive/My Drive/Oa_Suarez/Yelp_Tip_Business.csv', index=False)

cambio_tip_Ml = combined_df_business_tip

# creamos un CSV mas pequeño para el Machine Learning de Business y Checkin
columnas_a_mantener = ['business_id', 'text', 'state', 'categories']
cambio_tip_Ml = cambio_tip_Ml[columnas_a_mantener]
cambio_tip_Ml.to_csv('/content/drive/My Drive/Oa_Suarez/Yelp_Tip_Business-ML.csv', index=False)

# Sube el primer archivo CSV a Google Cloud Storage
blob7 = bucket.blob('Yelp_Tip_Business_PowerBI.csv')
blob7.upload_from_filename('/content/drive/My Drive/Oa_Suarez/Yelp_Tip_Business.csv')

# Sube el segundo archivo CSV a Google Cloud Storage
blob8 = bucket.blob('Yelp_Tip_Business-ML.csv')
blob8.upload_from_filename('/content/drive/My Drive/Oa_Suarez/Yelp_Tip_Business-ML.csv')



# -----------------------------------------------------------------------------------------------------------------------



archivo_parquet = '/content/drive/MyDrive/Oa_Suarez/Yelp/user.parquet'
tabla_parquet = pq.read_table(archivo_parquet)
user = tabla_parquet.to_pandas()
user = user.replace('', pd.NA)

combined_df_tip_user = user.merge(combined_df_business_tip, on='user_id', how='inner')

combined_df_tip_user.to_csv('/content/drive/My Drive/Oa_Suarez/Yelp_User_Tip.csv', index=False)

#lo guardo en un CSV
combined_df_tip_user.to_csv('/content/drive/My Drive/Oa_Suarez/Yelp_User_Tip.csv', index=False)

# creamos un CSV mas pequeño para el Machine Learning de User y Tip
#columnas_a_mantener = ['escriba las columnas que desea tener']
#cambio_use_Ml = cambio_tip_Ml[columnas_a_mantener]
#cambio_use_Ml.to_csv('/content/drive/My Drive/Oa_Suarez/Yelp_User_Tip-ML.csv', index=False)

# Sube el primer archivo CSV a Google Cloud Storage
blob9 = bucket.blob('Yelp_User_Tip_PowerBI.csv')
blob9.upload_from_filename('/content/drive/My Drive/Oa_Suarez/Yelp_User_Tip.csv')

# Sube el segundo archivo CSV a Google Cloud Storage
#blob10 = bucket.blob('Yelp_User_Tip-ML.csv')
#blob10.upload_from_filename('/content/drive/My Drive/Oa_Suarez/Yelp_User_Tip-ML.csv')

In [35]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
